## Lab 9: Predicting Forest Cover Type with SVMs

Author: Eva Newby

Collaborators: Emma Bea Mitchell, Michelle Yiv

### Introduction
In this lab, we will explore the application of Support Vector Machines (SVMs) and Random Forests (RFs) for multi-class classification using cartographic variables. Specifically, we will predict forest cover type based on a variety of environmental features such as elevation, soil type, and land aspect.

Understanding forest cover classification is crucial for natural resource management. Land managers and conservationists rely on accurate predictions of vegetation types to make informed decisions about wildlife habitats, fire management, and sustainable forestry practices. However, direct field assessments of forest cover can be costly and time-consuming, making predictive models a valuable tool for estimating cover types in large or inaccessible regions.

Dataset info here: https://archive.ics.uci.edu/dataset/31/covertype


### Step 0: Load Libraries and Data

In [1]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("/courses/EDS232/Data/covtype_sample.csv")

### Step 1: Data Preprocessing 

Before building our classification models, we need to prepare the dataset by separating the features target variable (`Cover_Type`) and  splitting the data into training and test sets. 

We didn't explicitly discuss it in lecture, but SVMs are sensitive to feature scale.  Use `describe()` to summarize the dataset.  Do you see anything that would require scaling of the data?  If so, apply that transformation.

In [2]:
# Summarize data
print(df.describe())

          Elevation        Aspect         Slope  \
count  10000.000000  10000.000000  10000.000000   
mean    2955.599500    154.450000     14.114700   
std      281.786673    111.851861      7.499705   
min     1860.000000      0.000000      0.000000   
25%     2804.750000     58.000000      9.000000   
50%     2995.000000    126.000000     13.000000   
75%     3159.000000    258.000000     18.000000   
max     3846.000000    359.000000     65.000000   

       Horizontal_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
count                      10000.000000                     10000.000000   
mean                         268.097600                        45.755300   
std                          211.899673                        58.034207   
min                            0.000000                      -164.000000   
25%                           95.000000                         7.000000   
50%                          218.000000                        29.000000   
75%     

Do you see anything that would require scaling of the data?  If so, apply that transformation.

Yes, some of the features (like distances versus soil type) have different scales. 

In [3]:
# define features and target
X = df.drop(columns=['Cover_Type'])
y = df['Cover_Type'] 

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=808)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Step 2: Hyperparameter Tuning for SVM
To optimize our SVM model, we need to search for the best hyperparameters that maximize classification accuracy. Since SVM performance depends heavily on `C`, `kernel`, and `gamma`, we will use `GridSearchCV()` to systematically test different combinations. Initialize a cross validation object with 5 folds using `StratifiedKFold`. Check out how `StratifiedKFold` differs from `Kfold` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html). 

Then, set up a grid to test different values of: 
- `C` (regularization strength): how strictly the model fits the training data
  - Candidate parameter values: `(0.1, 1, 10, 100)`
- `kernel` (decision boundary shape): compares linear and radial basis function shapes
  - Candidate parameter values: (linear, rbf)
- `gamma` (influence of training observations): influence of individual points on decision boundary
  - Candidate parameter values: (scale, auto)

As models and datasets become more complex, consideration of computation time becomes more important.  You'll use `time.time()` to measure the time required to fit the grid object.  

**Print the best parameters from your model, as well as the time required to fit the grid object.** 

In [ ]:
# Initialize model
svm = SVC()

# Initialize cv with stratified kfold
cv = StratifiedKFold(n_splits=5)

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],             
    'kernel': ['linear', 'rbf'],       
    'gamma': ['scale', 'auto']}

# different combos of C, Kernel, and gamma
grid_search_svm = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)

# Measure time 
start_time = time.time()
grid_search_svm.fit(X_train, y_train)
end_time = time.time()

# Print the best parameters
print("Best Parameters for SVM:", grid_search_svm.best_params_)
print(f"Time to fit GridSearchCV for SVM: {end_time - start_time:.2f} seconds")

### Step 3: Build and fit a Random Forest for comparison

Let's compare our SVM to a Random Forest classifier.  Create a grid for cross-validation with three hyperparameters of your choice to tune, along with three sensible values for each one.  
**Print the best parameters from your model, as well as the time required to fit the grid object.** 

In [ ]:
# Initalize Random Forest model
rf = RandomForestClassifier(random_state=808)

# Define hyperparameter grid, used lab 6 for reference
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]}

# Perform gridsearch cv
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=cv, scoring='accuracy', n_jobs=-1)

# Measure time 
start_time = time.time()
grid_search_rf.fit(X_train, y_train)
end_time = time.time()

# Print the best parameters 
print("Best Parameters for Random Forest:", grid_search_rf.best_params_)
print(f"Time to fit GridSearchCV: {end_time - start_time:.2f} seconds")

### Step 4: Model Predictions and Evaluation
Now that you have trained and optimized both a SVM and RF model, you will evaluate their performances on the test set to prepare for model comparison. In this step, you will:
- Use the best models from `GridSearchCV()` to make predictions on the test set
- Generate a confusion matrix for each model to visualize classification performance


In [ ]:
# Get the best models
best_svm = grid_search_svm.best_estimator_
best_rf = grid_search_rf.best_estimator_

# Make predictions on test set
y_pred_svm = best_svm.predict(X_test)
y_pred_rf = best_rf.predict(X_test)

# make confusion matrix for each model
cm_svm = confusion_matrix(y_test, y_pred_svm)
cm_rf = confusion_matrix(y_test, y_pred_rf)

In [ ]:
# Make confusion matrix for SVM
# Initialize figure
plt.figure(figsize=(12,6))

# Plot SVM confusion matrix
plt.subplot(1, 2, 1)
sns.heatmap(cm_svm, annot=True, fmt="d", cmap="Blues", cbar=False)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - SVM")

# Plot Random Forest confusion matrix
plt.subplot(1, 2, 2)
sns.heatmap(cm_rf, annot=True, fmt="d", cmap="Greens", cbar=False)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Random Forest")

plt.show()

### Step 5: Gather and display additional performance metrics
Now display the accuracy score and training time required for each model to so we can compare the models.

In [ ]:
# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Print accuracy
print(f"SVM Accuracy: {accuracy_svm:.4f}")
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

# Calculate SVM training time
start_time = time.time()
best_svm.fit(X_train, y_train)
svm_training_time = time.time() - start_time

# Random Forest training time
start_time = time.time()
best_rf.fit(X_train, y_train)
rf_training_time = time.time() - start_time

# Print training times
print(f"SVM Training Time (final model): {svm_training_time:.2f} seconds")
print(f"Random Forest Training Time (final model): {rf_training_time:.2f} seconds")

### Step 6: Compare the models
Now that we have trained, optimized, and evaluated both SVM and RF models, we will compare them based on overall accuracy, training time, and types of errors made.

Based on these comparisons, which model is more suitable for this task?  


Based on the numbers above, the random forest model outperforms the SVM model in accuracy and training time. 